<a href="https://colab.research.google.com/github/Toxly313/dashboard/blob/test-split/Kopie_von_informatik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Gold-Analyse mit Candlestick-Charts - Korrigierte Version
Candlestick-Chart wird jetzt garantiert angezeigt
"""

print("📦 Installiere benötigte Bibliotheken...")
!pip install yfinance pandas matplotlib plotly numpy --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotheken installiert!")

# Plotly für Colab konfigurieren
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio
pio.renderers.default = 'colab'  # Wichtig für Google Colab

# ----------------------------- KONFIGURATION --------------------------------
TICKER = "GC=F"  # Gold Futures in USD
CURRENCY = "USD"  # Währung
PERIOD = "6mo"    # Kürzerer Zeitraum für bessere Performance
INTERVAL = "1d"   # Tagesdaten

# ----------------------------- DATEN LADEN ----------------------------------
print(f"📥 Lade Gold-Daten für {TICKER} in {CURRENCY}...")

try:
    # Gold-Daten von yfinance laden
    gold_data = yf.download(TICKER, period=PERIOD, interval=INTERVAL, progress=False)

    if gold_data.empty:
        raise Exception("Keine Daten erhalten")

    print(f"✅ Erfolgreich {len(gold_data)} Datensätze geladen")
    print(f"📅 Zeitraum: {gold_data.index[0].strftime('%d.%m.%Y')} - {gold_data.index[-1].strftime('%d.%m.%Y')}")

except Exception as e:
    print(f"❌ Fehler beim Laden der Daten: {e}")
    print("💡 Verwende Fallback-Daten...")
    # Fallback: Simulierte Daten erstellen
    dates = pd.date_range(end=datetime.now(), periods=180, freq='D')
    base_price = 1800
    price_changes = np.random.normal(0, 15, 180)
    prices = base_price + np.cumsum(price_changes)

    gold_data = pd.DataFrame({
        'Open': prices * 0.998 + np.random.normal(0, 5, 180),
        'High': prices * 1.015 + np.random.normal(0, 3, 180),
        'Low': prices * 0.985 + np.random.normal(0, 3, 180),
        'Close': prices,
        'Volume': np.random.randint(80000, 300000, 180)
    }, index=dates)

# Daten bereinigen
gold_data = gold_data.dropna()

# ----------------------------- EINFACHER CANDLESTICK CHART ------------------
print("📈 Erstelle einfachen Candlestick-Chart...")

# Einfacher Candlestick Chart ohne zusätzliche Indikatoren (für zuverlässige Anzeige)
candle_fig = go.Figure(data=[go.Candlestick(
    x=gold_data.index,
    open=gold_data['Open'],
    high=gold_data['High'],
    low=gold_data['Low'],
    close=gold_data['Close'],
    name='Gold Preis',
    increasing_line_color='#00FF00',  # Grün für steigende Kerzen
    decreasing_line_color='#FF0000'   # Rot für fallende Kerzen
)])

candle_fig.update_layout(
    title=f'🪙 Gold Candlestick Chart - {TICKER} in {CURRENCY}',
    xaxis_title='Datum',
    yaxis_title=f'Preis ({CURRENCY})',
    xaxis_rangeslider_visible=True,  # Range Slider für Zoom
    height=500,
    template='plotly_dark',
    showlegend=True
)

# Aktuellen Preis markieren
current_price = gold_data['Close'].iloc[-1]
candle_fig.add_hline(
    y=current_price,
    line_dash="dash",
    line_color="yellow",
    annotation_text=f"Aktuell: {float(current_price):.2f} {CURRENCY}"
)

print("🔄 Zeige Candlestick-Chart an...")
candle_fig.show()

# ----------------------------- ERWEITERTER CHART MIT INDIKATOREN ------------
print("📊 Erstelle erweiterten Chart mit technischen Indikatoren...")

# Erweiterte Figur mit Subplots
advanced_fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(
        f'Gold Candlestick Chart - {TICKER}',
        'RSI (Relative Strength Index)',
        'MACD'
    ),
    row_heights=[0.6, 0.2, 0.2]
)

# 1. Candlestick im oberen Chart
advanced_fig.add_trace(
    go.Candlestick(
        x=gold_data.index,
        open=gold_data['Open'],
        high=gold_data['High'],
        low=gold_data['Low'],
        close=gold_data['Close'],
        name='Gold',
        increasing_line_color='green',
        decreasing_line_color='red'
    ), row=1, col=1
)

# Einfache gleitende Durchschnitte hinzufügen
gold_data['SMA_20'] = gold_data['Close'].rolling(window=20).mean()
gold_data['SMA_50'] = gold_data['Close'].rolling(window=50).mean()

advanced_fig.add_trace(
    go.Scatter(
        x=gold_data.index,
        y=gold_data['SMA_20'],
        line=dict(color='orange', width=1),
        name='SMA 20'
    ), row=1, col=1
)

advanced_fig.add_trace(
    go.Scatter(
        x=gold_data.index,
        y=gold_data['SMA_50'],
        line=dict(color='blue', width=1),
        name='SMA 50'
    ), row=1, col=1
)

# 2. RSI berechnen und hinzufügen
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

gold_data['RSI'] = calculate_rsi(gold_data['Close'])

advanced_fig.add_trace(
    go.Scatter(
        x=gold_data.index,
        y=gold_data['RSI'],
        line=dict(color='purple', width=2),
        name='RSI'
    ), row=2, col=1
)

# RSI Grenzen
advanced_fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
advanced_fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
advanced_fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

# 3. MACD berechnen und hinzufügen
def calculate_macd(prices, fast=12, slow=26, signal=9):
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    macd_histogram = macd - macd_signal
    return macd, macd_signal, macd_histogram

macd, macd_signal, macd_histogram = calculate_macd(gold_data['Close'])
gold_data['MACD'] = macd
gold_data['MACD_Signal'] = macd_signal
gold_data['MACD_Histogram'] = macd_histogram

advanced_fig.add_trace(
    go.Scatter(
        x=gold_data.index,
        y=gold_data['MACD'],
        line=dict(color='blue', width=2),
        name='MACD'
    ), row=3, col=1
)

advanced_fig.add_trace(
    go.Scatter(
        x=gold_data.index,
        y=gold_data['MACD_Signal'],
        line=dict(color='red', width=2),
        name='MACD Signal'
    ), row=3, col=1
)

# MACD Histogramm
macd_hist_colors = ['green' if x >= 0 else 'red' for x in gold_data['MACD_Histogram'].fillna(0)]
advanced_fig.add_trace(
    go.Bar(
        x=gold_data.index,
        y=gold_data['MACD_Histogram'],
        name='MACD Histogram',
        marker_color=macd_hist_colors,
        opacity=0.6
    ), row=3, col=1
)

# Layout anpassen
advanced_fig.update_layout(
    title_text=f"Gold Technische Analyse - {TICKER} in {CURRENCY}",
    height=800,
    xaxis_rangeslider_visible=False,
    template='plotly_white',
    showlegend=True
)

print("🔄 Zeige erweiterten Chart an...")
advanced_fig.show()

# ----------------------------- KAUFEMPFEHLUNG GENERIEREN --------------------
print("🤖 Generiere Kaufempfehlung...")

# Aktuelle Werte sicher extrahieren
def safe_extract(series, default=0):
    try:
        if len(series) > 0:
            value = series.iloc[-1]
            if not np.isnan(value):
                return float(value)
        return default
    except:
        return default

current_price = safe_extract(gold_data['Close'], 1800)
current_rsi = safe_extract(gold_data['RSI'], 50)
current_macd = safe_extract(gold_data['MACD'], 0)
current_macd_signal = safe_extract(gold_data['MACD_Signal'], 0)
sma_20_current = safe_extract(gold_data['SMA_20'], current_price)
sma_50_current = safe_extract(gold_data['SMA_50'], current_price)

# Bewertungskriterien
signals = []
score = 0

# 1. Trend-Analyse
if sma_20_current > sma_50_current:
    signals.append("✅ Aufwärtstrend (SMA20 > SMA50)")
    score += 2
else:
    signals.append("❌ Abwärtstrend (SMA20 < SMA50)")

# 2. RSI Analyse
if current_rsi < 30:
    signals.append("✅ RSI zeigt überverkaufte Bedingungen")
    score += 2
elif current_rsi < 45:
    signals.append("⚠️ RSI im unteren neutralen Bereich")
    score += 1
elif current_rsi > 70:
    signals.append("❌ RSI zeigt überkaufte Bedingungen")
    score -= 2
else:
    signals.append("➡️ RSI im neutralen Bereich")

# 3. MACD Analyse
if current_macd > current_macd_signal:
    signals.append("✅ MACD über Signal-Linie (bullisch)")
    score += 1
else:
    signals.append("❌ MACD unter Signal-Linie (bärisch)")

# 4. Preis über SMA
if current_price > sma_20_current:
    signals.append("✅ Preis über SMA 20")
    score += 1
else:
    signals.append("❌ Preis unter SMA 20")

# Empfehlung basierend auf Score
if score >= 5:
    recommendation = "🚀 STARKE KAUFEMPFEHLUNG"
    confidence = "SEHR HOCH"
    color = "🟢"
elif score >= 3:
    recommendation = "📈 KAUFEMPFEHLUNG"
    confidence = "HOCH"
    color = "🟡"
elif score >= 1:
    recommendation = "⚖️ NEUTRAL / ABWARTEN"
    confidence = "MITTEL"
    color = "🟠"
else:
    recommendation = "📉 VERKAUFEN / ABWARTEN"
    confidence = "HOCH"
    color = "🔴"

# ----------------------------- ZUSAMMENFASSUNG ------------------------------
print("\n" + "="*70)
print("🎯 GOLD ANALYSE - ZUSAMMENFASSUNG & EMPFEHLUNG")
print("="*70)

print(f"💰 AKTUELLE DATEN ({CURRENCY}):")
print(f"   • Ticker: {TICKER}")
print(f"   • Aktueller Preis: {current_price:.2f} {CURRENCY}")
print(f"   • Zeitraum: {gold_data.index[0].strftime('%d.%m.%Y')} - {gold_data.index[-1].strftime('%d.%m.%Y')}")

print(f"\n📊 TECHNISCHE INDIKATOREN:")
print(f"   • RSI (14): {current_rsi:.1f}")
print(f"   • MACD: {current_macd:.4f}")
print(f"   • MACD Signal: {current_macd_signal:.4f}")
print(f"   • SMA 20: {sma_20_current:.2f}")
print(f"   • SMA 50: {sma_50_current:.2f}")

print(f"\n📈 SIGNALANALYSE:")
for signal in signals:
    print(f"   {signal}")

print(f"\n{color} EMPFEHLUNG: {recommendation}")
print(f"   📊 Konfidenzniveau: {confidence}")
print(f"   🎯 Score: {score}/6 Punkten")

print(f"\n💡 HANDLUNGSEMPFEHLUNG:")
if score >= 4:
    print("   ✓ Guter Einstiegszeitpunkt")
    print("   ✓ Trend und Momentum positiv")
    print("   ✓ Stop-Lose setzen und Position aufbauen")
elif score >= 2:
    print("   ⚠️  Vorsichtige Kaufempfehlung")
    print("   ⚠️  Auf Bestätigung warten")
    print("   ⚠️  Nur kleine Positionen")
else:
    print("   ✗ Derzeit kein Kauf empfohlen")
    print("   ✗ Abwarten oder alternative Anlagen")
    print("   ✗ Auf bessere Einstiege warten")

print(f"\n⚠️  WICHTIGER HINWEIS:")
print("   Diese Analyse dient Bildungszwecken und")
print("   stellt KEINE Finanzberatung dar!")
print("="*70)

# ----------------------------- VOLUMEN-CHART --------------------------------
print("\n📊 Erstelle Volumen-Chart...")

# Volumen-Chart
volume_fig = go.Figure()

# Farben für Volumen-Bars (grün wenn Preis gestiegen, rot wenn gefallen)
colors = []
for i in range(len(gold_data)):
    if gold_data['Close'].iloc[i] > gold_data['Open'].iloc[i]:
        colors.append('green')
    else:
        colors.append('red')

volume_fig.add_trace(go.Bar(
    x=gold_data.index,
    y=gold_data['Volume'],
    name='Handelsvolumen',
    marker_color=colors,
    opacity=0.7
))

volume_fig.update_layout(
    title='Handelsvolumen für Gold',
    xaxis_title='Datum',
    yaxis_title='Volumen',
    height=400,
    template='plotly_white'
)

volume_fig.show()

print("\n✅ Analyse abgeschlossen!")
print("📈 Zwei Candlestick-Charts wurden angezeigt:")
print("   1. Einfacher Candlestick-Chart")
print("   2. Erweiterter Chart mit technischen Indikatoren")
print("📊 Volumen-Chart zeigt Handelsaktivität")

📦 Installiere benötigte Bibliotheken...
✅ Bibliotheken installiert!
📥 Lade Gold-Daten für GC=F in USD...
✅ Erfolgreich 130 Datensätze geladen
📅 Zeitraum: 30.04.2025 - 31.10.2025
📈 Erstelle einfachen Candlestick-Chart...


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().